In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv


In [2]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, callbacks, utils, optimizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import gradio as gr

2025-10-09 10:35:51.967554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760006152.227426      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760006152.298062      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Cellule 2 - Chargement et aperçu des données
data_path = '/kaggle/input/french-twitter-sentiment-analysis/french_tweets.csv'
data = pd.read_csv(data_path)

# Aperçu
display(data.head())
print("\nInfo :")
print(data.info())
print("\nDistribution originale des labels :")
print(data['label'].value_counts())

,label,text
0,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
1,0,Est contrarié qu'il ne puisse pas mettre à jou...
2,0,J'ai plongé plusieurs fois pour la balle. A ré...
3,0,Tout mon corps a des démangeaisons et comme si...
4,0,"Non, il ne se comporte pas du tout. je suis en..."



Info :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526724 entries, 0 to 1526723
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   label   1526724 non-null  int64 
 1   text    1526724 non-null  object
dtypes: int64(1), object(1)
memory usage: 23.3+ MB
None

Distribution originale des labels :
label
0    771604
1    755120
Name: count, dtype: int64


In [4]:
TARGET_PER_CLASS = 600000
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

def safe_sample_per_class(df, label_col='label', n=TARGET_PER_CLASS, random_state=SEED):
    parts = []
    for lbl, group in df.groupby(label_col):
        sample_n = min(len(group), n)
        parts.append(group.sample(n=sample_n, random_state=random_state))
    return pd.concat(parts).reset_index(drop=True)

data_balanced = safe_sample_per_class(data, label_col='label', n=TARGET_PER_CLASS)
print("Nouvelle distribution (après échantillonnage sécurisé) :")
print(data_balanced['label'].value_counts())

Nouvelle distribution (après échantillonnage sécurisé) :
label
0    600000
1    600000
Name: count, dtype: int64
